## Урок 7. Рекуррентные нейронные сети RNN LSTM GRU


### Задание:  
На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конкурируещими. Выяснить, какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара.

 1. построить свёрточные архитектуры  
 2. построить различные архитектуры с RNN
 3. построить совместные архитектуры CNN -> RNN и (RNN -> CNN)
 4. сделать выводы что получилось лучше





#### Решение

In [1]:
!pip install pymorphy2
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 4.3 MB/s 
     |████████████████████████████████| 8.2 MB 60.5 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=0654e32e5cf3b2a0dcef0071b8cfec256d8ea434223e2771f3b4b4a76128e4c2
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=d4637522849d0a71bbffb14fe1599d2631399df42b40005833545bdb569cd22c
  Stored in directory: /root/.cache/pip/wheels/eb/03/0d/3bd31c983789aeb0b4d5e2ca48590288d9db1586cf5f225062
Successfully built stop-words


In [2]:
import warnings
warnings.filterwarnings("ignore")

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
import re

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Conv1D, Activation, Masking, SimpleRNN, Dropout, LSTM, GRU, Flatten, MaxPooling1D
# from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
# Возьмем датасет с прошлого занятия

df_train = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/NLP_8/train.csv")
df_test = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/NLP_8/test.csv")
df_val = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/NLP_8/val.csv")

df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


#### Предобработка

In [9]:
exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()

    txt = re.sub("\sне", "не", txt)

    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    # txt = [word for word in txt if len(word)>1] # условие "более одного слова в тексте"

    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

#### Токенизация

In [10]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [11]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

x_train = pad_sequences(sequences_train, maxlen=training_length)
x_val = pad_sequences(sequences_val, maxlen=training_length)

In [12]:
y_train = df_train['class'].values
y_val = df_val['class'].values

Накопление результатов

In [13]:
# накопление результатов
result = []

1.CNN

In [14]:
def model_CNN(word_count=258108, training_length=27, batch_size=512, epochs=10):
  model = tf.keras.Sequential()
  model.add(Embedding(input_dim=word_count, 
                      output_dim=128, 
                      input_length=training_length))
  model.add(Conv1D(128, 3, activation='relu'))
  model.add(GlobalMaxPooling1D())
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  # компиляция модели
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  
  model.summary()

  # обучение модели
  early_stopping=EarlyStopping(monitor='val_loss')  
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
  
  # оценка качества модели
  [loss, accuracy] = model.evaluate(x_val, y_val)
  print("Test loss {:.4f}, accuracy {:.2f}%".format(loss, accuracy * 100))

  # накопление результатов
  result.append(['CNN', loss, accuracy])

  return result

In [15]:
model_CNN(word_count=word_count, training_length=training_length, batch_size=512, epochs=10)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 27, 128)           33037824  
                                                                 
 conv1d (Conv1D)             (None, 25, 128)           49280     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 33,088,405
Trainable params: 33,088,405
Non-trainable params: 0
____________________________________________

[['CNN', 0.6070356965065002, 0.7350879311561584]]

2.SimpleRNN

In [16]:
def model_simple_RNN(word_count=258108, training_length=27, batch_size=512, epochs=10):
  model = tf.keras.Sequential()

  model.add(
      Embedding(input_dim=word_count,
                input_length=training_length,
                output_dim=30,
                trainable=True,
                mask_zero=True))
  model.add(Masking(mask_value=0.0))

  model.add(SimpleRNN(64))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  # компиляция модели
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  
  model.summary()

  # обучение модели
  early_stopping=EarlyStopping(monitor='val_loss')  
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
  
  # оценка качества модели
  [loss, accuracy] = model.evaluate(x_val, y_val)
  print("Test loss {:.4f}, accuracy {:.2f}%".format(loss, accuracy * 100))

  # накопление результатов
  result.append(['Simple_RNN', loss, accuracy])

  return result

In [17]:
model_simple_RNN(word_count=word_count, training_length=training_length, batch_size=512, epochs=10)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 masking (Masking)           (None, 27, 30)            0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                6080      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 7,753,545
Trainable params: 7,753,545
No

[['CNN', 0.6070356965065002, 0.7350879311561584],
 ['Simple_RNN', 0.5957046151161194, 0.7312965393066406]]

3.LSTM с шириной слоя LSTM 128

In [18]:
def model_LSTM(word_count=258108, training_length=27, batch_size=512, epochs=10):
  model = tf.keras.Sequential()

  model.add(
      Embedding(input_dim=word_count,
                input_length=training_length,
                output_dim=30,
                trainable=True,
                mask_zero=True))
  model.add(Masking(mask_value=0.0))
  model.add(LSTM(128, recurrent_dropout=0.2))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  # компиляция модели
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  
  model.summary()

  # обучение модели
  early_stopping=EarlyStopping(monitor='val_loss')  
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
  
  # оценка качества модели
  [loss, accuracy] = model.evaluate(x_val, y_val)
  print("Test loss {:.4f}, accuracy {:.2f}%".format(loss, accuracy * 100))

  # накопление результатов
  result.append(['LSTM', loss, accuracy])

  return result

In [19]:
model_LSTM(word_count=word_count, training_length=training_length, batch_size=512, epochs=10)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 masking_1 (Masking)         (None, 27, 30)            0         
                                                                 
 lstm (LSTM)                 (None, 128)               81408     
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 7,832,969
Trainable params: 7,832,969
No

[['CNN', 0.6070356965065002, 0.7350879311561584],
 ['Simple_RNN', 0.5957046151161194, 0.7312965393066406],
 ['LSTM', 0.5865833163261414, 0.7393642663955688]]

4.GRU с шириной слоя GRU 128

In [20]:
def model_GRU(word_count=258108, training_length=27, batch_size=512, epochs=10):
  model = tf.keras.Sequential()

  model.add(
      Embedding(input_dim=word_count,
                input_length=training_length,
                output_dim=30,
                trainable=True,
                mask_zero=True))
  model.add(Masking(mask_value=0.0))
  model.add(GRU(128, recurrent_dropout=0.2))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  # компиляция модели
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  
  model.summary()

  # обучение модели
  early_stopping=EarlyStopping(monitor='val_loss')  
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
  
  # оценка качества модели
  [loss, accuracy] = model.evaluate(x_val, y_val)
  print("Test loss {:.4f}, accuracy {:.2f}%".format(loss, accuracy * 100))

  # накопление результатов
  result.append(['GRU', loss, accuracy])

  return result

In [21]:
model_GRU(word_count=word_count, training_length=training_length, batch_size=512, epochs=10)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 masking_2 (Masking)         (None, 27, 30)            0         
                                                                 
 gru (GRU)                   (None, 128)               61440     
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 7,813,001
Trainable params: 7,813,001
No

[['CNN', 0.6070356965065002, 0.7350879311561584],
 ['Simple_RNN', 0.5957046151161194, 0.7312965393066406],
 ['LSTM', 0.5865833163261414, 0.7393642663955688],
 ['GRU', 0.5692830085754395, 0.7381298542022705]]

5.CNN -> RNN

In [22]:
def model_CNN_RNN(word_count=258108, training_length=27, batch_size=512, epochs=10):
  model = tf.keras.Sequential()

  model.add(
      Embedding(input_dim=word_count,
                input_length=training_length,
                output_dim=30,
                trainable=True,
                mask_zero=True))
  model.add(Conv1D(64, 2, padding='same', activation='relu'))
  model.add(MaxPooling1D(2, padding='same'))
  model.add(Conv1D(32, 2, padding='same', activation='relu'))
  model.add(MaxPooling1D(2, padding='same'))


  model.add(LSTM(64, return_sequences=True, recurrent_dropout = 0.2))
  model.add(LSTM(32))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  # компиляция модели
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  
  model.summary()

  # обучение модели
  early_stopping=EarlyStopping(monitor='val_loss')  
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
  
  # оценка качества модели
  [loss, accuracy] = model.evaluate(x_val, y_val)
  print("Test loss {:.4f}, accuracy {:.2f}%".format(loss, accuracy * 100))

  # накопление результатов
  result.append(['CNN_RNN', loss, accuracy])

  return result

In [23]:
model_CNN_RNN(word_count=word_count, training_length=training_length, batch_size=512, epochs=10)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 conv1d_1 (Conv1D)           (None, 27, 64)            3904      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 14, 64)           0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 14, 32)            4128      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 7, 32)            0         
 1D)                                                             
                                                                 
 lstm_1 (LSTM)               (None, 7, 64)            

[['CNN', 0.6070356965065002, 0.7350879311561584],
 ['Simple_RNN', 0.5957046151161194, 0.7312965393066406],
 ['LSTM', 0.5865833163261414, 0.7393642663955688],
 ['GRU', 0.5692830085754395, 0.7381298542022705],
 ['CNN_RNN', 0.5835556387901306, 0.736234188079834]]

Результат

In [28]:
pd.DataFrame(result, columns=['model', 'loss', 'accuracy'
                              ]).\
                              sort_values('loss',ascending=True).head(6)

,model,loss,accuracy
3,GRU,0.569283,0.738130
4,CNN_RNN,0.583556,0.736234
2,LSTM,0.586583,0.739364
1,Simple_RNN,0.595705,0.731297
0,CNN,0.607036,0.735088
